Different morphologies of the same neuron type might require different activation to reach comparable firing frequency. InputScaling simulates a network of a neuron type with different number of inputs, and different frequency to infer what level of input is needed.

Here we look at the input to ```neuron_types="fs"```.

During a simulation we can vary the input frequency, but not the number of inputs coupled to a neuron, so we use a set of ```num_replicas``` copies of each morphology, each receiving a different number of inputs from ```num_input_min``` to ```num_input_max``` of a given ```input_type``` (e.g. for striatum "cortical" or "thalamic"). Note that if 100 inputs are requested, then on average 100 inputs will be placed, but the number can vary as the placement is random.

A set of input frequencies are tested, specified by ```input_frequency_range```, and each of these frequencies is run for ```input_duration``` seconds. In this example we create a small example, but we recommend that use use a longer input duration (e.g. 10 seconds, or possibly more).

The user can specify ```parameter_key```, ```morphology_key``` and (neuron) ```modulation_key``` to be used for the neuron model.

The code below assumes you have ```BasalGangliaData``` repository installed.

In [1]:
import os
from snudda.input.input_tuning import InputTuning

os.environ["SNUDDA_DATA"] = "../../../BasalGangliaData/data/"
assert os.path.isdir(os.getenv("SNUDDA_DATA")), f"You need to have BasalGangliaData installed for this example."

network_path = os.path.join("networks", "input_tuning")
input_tuning = InputTuning(network_path)

Reading SNUDDA_DATA=../../../BasalGangliaData/data/ from environment variable $SNUDDA_DATA


We create a network without any synapses and connect different number of external inputs to the neurons. Each neuron then receives a range of input frequencies. Here ```neurons_path``` points to the folder that contains all the different neuron types. Note that we use ```$DATA``` to refer to the ```snudda/data``` folder, but you can use an absolute path instead if you have your own neurons folder. If you use your own neurons you probably want to compile the neuron mechanisms from another folder when calling ```nrnivmodl``` below.

Also note that the folder names for the different neurons must be lowercase.

In [3]:
neurons_path = os.path.join("$DATA", "neurons", "striatum")
input_tuning.setup_network(neurons_path=neurons_path, 
                           num_replicas=4,
                           neuron_types="fs")
input_tuning.setup_input(input_type="cortical",  # eg. "cortical" or "thalamic"
                         num_input_min=50,
                         num_input_max=200,
                         input_duration=3.0,
                         input_frequency_range=[1.0, 2.0])

Skipping neuron type dspn
Found 149 neuron models in /home/hjorth/HBP/BasalGangliaData/data/neurons/striatum/fs
Skipping neuron type chin
Skipping neuron type ispn
Skipping neuron type lts
Writing network config file to networks/input_tuning/network-config.json
Reading SNUDDA_DATA=None from networks/input_tuning/network-config.json
Reading SNUDDA_DATA=../../../BasalGangliaData/data/ from environment variable $SNUDDA_DATA
Reading SNUDDA_DATA=None from networks/input_tuning/network-config.json
Reading SNUDDA_DATA=../../../BasalGangliaData/data/ from environment variable $SNUDDA_DATA
No d_view specified, running distribute neurons in serial
No connections specified in connectivity_distribution.
Using cortical synapse density for input.
Reading SNUDDA_DATA=None from networks/input_tuning/network-config.json
Reading SNUDDA_DATA=../../../BasalGangliaData/data/ from environment variable $SNUDDA_DATA
Loading 596 neurons with 0 synapses and 0 gap junctions
Writing spikes to networks/input_tunin

In [4]:
!nrnivmodl ../../../BasalGangliaData/data/neurons/mechanisms

/home/hjorth/HBP/Snudda/examples/notebooks
Mod files: "../../../BasalGangliaData/data/neurons/mechanisms/../../../BasalGangliaData/data/neurons/mechanisms/bk_ch.mod" "../../../BasalGangliaData/data/neurons/mechanisms/../../../BasalGangliaData/data/neurons/mechanisms/bk_fs.mod" "../../../BasalGangliaData/data/neurons/mechanisms/../../../BasalGangliaData/data/neurons/mechanisms/bk_ms.mod" "../../../BasalGangliaData/data/neurons/mechanisms/../../../BasalGangliaData/data/neurons/mechanisms/ca_ch.mod" "../../../BasalGangliaData/data/neurons/mechanisms/../../../BasalGangliaData/data/neurons/mechanisms/cadyn_fs.mod" "../../../BasalGangliaData/data/neurons/mechanisms/../../../BasalGangliaData/data/neurons/mechanisms/cadyn_ms.mod" "../../../BasalGangliaData/data/neurons/mechanisms/../../../BasalGangliaData/data/neurons/mechanisms/cal12_ms.mod" "../../../BasalGangliaData/data/neurons/mechanisms/../../../BasalGangliaData/data/neurons/mechanisms/cal13_ms.mod" "../../../BasalGangliaData/data/neuron

## For large runs, to simulate in parallel use:

```mpiexec -n 4 python3 ../../snudda/input/input_tuning.py simulate networks/input_tuning/```

If you have more than four cores available on your machine, increase the number of workers accordingly.

## Below we just run it in serial in the notebook:

In [ ]:
input_tuning.simulate()

## Analyse results

In [ ]:
%matplotlib notebook

input_tuning.analyse_results(show_plots=True)